In [ ]:
#import
import pandas as pd
import numpy as np
import hddm
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import pymc
import kabuki
sns.set(style="white")
%matplotlib inline
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [ ]:
contdata = pd.read_csv("RLDDMS1_contingent.csv")
contdata.head()

In [ ]:
contdata = hddm.utils.flip_errors(contdata)

fig = plt.figure()
ax = fig.add_subplot(111, xlabel = 'RT', ylabel = 'count', title = 'RT distribution')
for i, subj_data in contdata.groupby('subj_idx'):
    subj_data.rt.hist(bins = 20, histtype = 'step', ax = ax)
    
    plt.savefig('Reactions times Contingent.pdf')

In [ ]:
from kabuki.analyze import gelman_rubin

models = []
for i in range (3):
    DDMCon = hddm.HDDM(contdata, depends_on = {'a': ['stim'], 't': ['stim'], 'v': ['stim']}, p_outlier = 0.05, bias = False)
    DDMCon.find_starting_values()
    DDMCon.sample(40000, burn = 30000, dbname = 'traces.db', db = 'pickle')
    models.append(DDMCon) 
    
gelman_rubin(models)

In [ ]:
M09_concat = kabuki.utils.concat_models(models)

In [ ]:
np.max(list(gelman_rubin(models).values()))

In [ ]:
M09_concat.print_stats()

In [ ]:
M09_concat.plot_posteriors()

In [ ]:
M09_traces = M09_concat.get_traces()

In [ ]:
M09_traces.to_csv(r'C:\Users\Hubsc\OneDrive\Desktop\DiscriminationStudy\M10_Draws.csv', index = False, header = True, sep = ';')